<a href="https://colab.research.google.com/github/T-Sunm/Learn-pandas/blob/main/Exercise_Renaming_and_Combining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'pitchfork-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F655%2F1252%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240820%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240820T134430Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D44c53e768c6cc578426aeefddc90a2fd1cec78017c4ac959625e18ae93e425226e72bfbfb7b18414817c2416e63e83f6ce14b11dd57338c9aae9eba9bcd52d6161d8e94f89c2b0f0f29081b3adeb7964baf61a3c82c4640fe0a72139c365ce675c4f39e23a147a324cd60a75c1dae440753c59bf368f7bde3f83c2367eaf582ecd3581457f72c25833ed0bcb1b2366068459eddb1035518d1f41f700de789fd9b945344635180213489ea54ba53816f0434550840b38d3282ebaad0423c4b5f2810d98d908ebfcc82155ac25eb108c171a530bb32310384c2b548129fde88dcd4788455f5283bc5713a466702bf418eda862056fd67178f6f5a889ec150a352a,chess:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2321%2F3919%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240820%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240820T134430Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D683c409268c3d034f72f3cf668b3808b090fa850274904ad803091f30b2069ad6faf48050b7e8768fcafab812f3b8fb00b6e08f0190f2217063f1b8749ba139e3fac7a7ca7cf95086fc6dd450142df149a5d34dda468d875b2622202a5cac679364291622f4422f6d428ff3112d95fec4783dc847ac3e85bdc68c4c0408db9ff80babd6cf0e1f765ed4ee0573ab2ebc80ee44d02ca03baed7d07ed0b552f6a4e3fbeae2d2b4abf2bd2fdf15df6c62f0f4fbd2c38ac785a42c9a7b00c4fc1c82373c03be9dd7a54839d0930f64d13f01819a10dac0ec64161ec074ad8512a999271e9d296df6581b83d04a128bec4dca8aeadd4284d1a7d8f260c911fcaba6d1d,kepler-exoplanet-search-results:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2894%2F4877%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240820%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240820T134430Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D67be33a88de3843ab9c52c91fadc7ab795f476b1fd313a1bdfbc8a4808a607f758a67babcb0705b27093d460d89285db80f37b42ffa90e7c13166220bcfd0bf8320af516a5b429880824f5e7a1aacaca5d48d13be78d03b05a5a295a0774fb4b406d63eac30b29d1d5851837a4a3f1e20f10b4ac42f6c95e2d12d63254a7a5a8ec37fb1c5e9eff2fdf39f2cbf38bcfd0ae2db7f3ad93db964d636614629d3911e524cd6a683671140eba72d4a35f0f7987189b68bbb8d5dcb6bbe7729248b10f1808e5fa1afa856b25471eab43dde465133d11fe0e432030c70ee65cd10f64df80d61847e5995ac86fb3bb26c35d7e3dc9886af4bb2730fda80db0c3a1394c05,things-on-reddit:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3491%2F5624%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240820%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240820T134430Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D29b8b953e6a01a95e56c8503467843c55406f55770efa2fcfb0408d63f0b754bdf67971d95fd018b86d8cc4d1cf027cfbbc31ca8fdf18f288b12f398b79b95fe34f72a267cedf681ca095ace80fdd8ed7f7058b701dca28b6c30c0cce1a034f67c3c5edc8b7fdd5280b8c10635eaaf0f24f0eeab02c1896d90311af39166818dbe41cb6e7df0333ffc0d6b9468d5e8fa2e288601a107521f0aa6dd311d5ae09061cfdc481e314d6e943078f75cf5826d2af043f3f63c5ef470604eb6ace0f0fafae27a398601886fa5df155c31cf376f859151f9cb44d5d0557d73609e3cbcf518fb12664612a54619d353a5427c17c7f8cc1fb4f3a99aee3dc22576f7d8c964,wine-reviews:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1442%2F8172%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240820%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240820T134430Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3e5c0148328ec2b627ad3cd2e17011b4400e4dff16af8e43d7fabd7e744dd90772d09e339d1c5a3f22f2a3d2ea4fae5cf1834b471eebe1389275d0dc6dfd0c23e20405e33c15aec53c5a30416d9e9987a0ab15a41d33e9f57e43f2cf7d860892529af374d35a3979acf0f6139c71e842c2bdaa5d6ae6d869c5468dfc4800d192a011168e6f14d18a34d6118b6a5f62bad0ce738ee25fea79a7042844c42c4daf44084ee72963f1bc95ba2cbcea4a8414704049772e3a5f1d2f63df35b628e82d02811e693e5d617545c48b79889894746cc81e142882a1a5e72340a8387c0ae57056fdcd86d7c8d6d4563237ed00cf6777b4f880dc9dca37d6ff00bab9203927,ramen-ratings:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F9366%2F13206%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240820%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240820T134430Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D60465e3c5094bab1abb726bb47fd4a3d54566ce3cd7947fda733bc7397db35a53d4a754643684f4e01fcf7f17189b505f67a9b308eb862068eaa721d8c37c79198a9c6a837dab2e1af42f630b2e811a689ae84b0aaed1919a6d5615082b37a4a98bcabd31655916a933aed0208cf6d754f3051a2c5170ad41f1265b1b7a57a73b3bfad3111ef4864ee1c23f60316eed5e0929fa763ba2624ed2e45d9eaee422d4489f7042501d2e3cbde49b121458cf0fb859749882ef05c29a5bc0ddc985b5afe10e069aa64104da2bc95958df48ccdba3b68513d43d47446ed9e914be5bb9e8e8671be33f88d0e8dca8539339640a279fd9d639cbd3d1a288f029814cf518f,powerlifting-database:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F179555%2F403916%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240820%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240820T134430Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4eb80a54d7a94750fa690cc7d7cfa9ed2dd7f5e236023f0b4ec3925af6490591d88c3116b2b05c7bdcf7ccbd47a758128d38b88a74eb41e5026780038662dd349ba1e864cdbdb206f91771378eeca439d6079b72a28bd106b2d9569a376b6daf8a39cecf4a032f467305566169607eaedc9af4035d66b61a97a848f8e6483f01723109d60ad7a244c572aae820c68f10eb288ba6bd2abfeeced145e1f94894c5be90dd52d275a08a07b2f7ded56123d8d9c5c39fd4230a709ef6f9386f1072f15b2f3bd6f111fc540e46996a2ee29d357a01c0ac754ea4c36997476cf20e9bada28deb89d1127167806eddf2f1ffc534aa33c555b2223955084fced4605be863,youtube-new:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4549%2F466349%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240820%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240820T134430Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1bf835fa84cfacf6c2bedb489a69f77f623efc6fea363ada420737d138ff7cc544deec4948ea021f7bd4b7341eb8c91f140d03b428253228da4c230a093e3cd879976d6d40216e2cf1cf99a95425ce8397143cfb70b73dd9742cbbd838874da2226c8954c083db986555e60ba387a30b604481af9ca89ed22c47d9ba90036d83d68a2918308a7e4eec61cfe7743860a1acc086a364cd1f45473e848c46130745727d168c117ae646e46cfd27591fefe5533d3dd008c4b9f0789124836bfc392b4855dfcce50182b9a344d46ec79c5cdccfe95e733a858084919ced60565868dbb5e7a9b3b250c620f00b7cba54cd1b823076079d297f52ed0fc675194b4268a8,188-million-us-wildfires:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2478%2F1151655%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240820%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240820T134430Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D457d0dfcc161c957c51f9c9030c7c1e1e1af2918c50f7b84fb5ee53b48bf979eba42079d7a765993964a2b9899b28e6572d23e6226efc4f13b847d5b6ed27b882595d3e11e1e0d4c03ed031902f98a7f3b2c931e95d4382e798d43782ba2d03089c3732476e0a14d50b0970162c6b983754ccbb4365e715ccf31c04ef50ad2d562394226b0fdba014318ecd86cb89882f15678f0369cef6aea95c1adbbcfb419c87279d279c28491a1c091fa06528a5305ba9addd0147b9103d283286406196495ca31e449994d1e7ac4ebd02d1c4fed10a2d9dd99aa8bf7e0040005c92c132afa9006ac388be7d5d2bb91b65b999294882c0b1480d43c52da9970df0ff54bd9,publicassistance:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F10128%2F5438389%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240820%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240820T134430Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D97570e37c7ade0b54bc716324541ee4ad64ffd0dc372e7b3926f7f3f24cef793f7aa73dd77cc741f2d0debb88b4a27963005b094ec3e1740d51eadcab8a84a9bdd4de0cec3649400acd7cdd8b2e5f96c61e60e68a2666cb824e17441da50c8fa22d0c22e956dcc45419b8547e544f1bbd8d78279b59e5930c5c050305dcf0e70271a9d91d4b5baef1106654016f319a2046f735133c6c2e20f5209f97673999be1cb4a960b2e985268257a1d5e2cbde9465d3b66a785b9ac82c6e78d759cf4dfdbc2b0446ebc421bde55580a4ed19f0a53ad16a947a36afceb6c15846b42cb51a6ed4f2e0a61a18219ff37884d64175f12397a3c79ae35d3fb6d64ee4533f295'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


**This notebook is an exercise in the [Pandas](https://www.kaggle.com/learn/pandas) course.  You can reference the tutorial at [this link](https://www.kaggle.com/residentmario/renaming-and-combining).**

---


# Introduction

Run the following cell to load your data and some utility functions.

In [ ]:
import pandas as pd

reviews = pd.read_csv("../input/wine-reviews/winemag-data-130k-v2.csv", index_col=0)

from learntools.core import binder; binder.bind(globals())
from learntools.pandas.renaming_and_combining import *
print("Setup complete.")

# Exercises

View the first several lines of your data by running the cell below:

In [ ]:
reviews.head()

## 1.
`region_1` and `region_2` are pretty uninformative names for locale columns in the dataset. Create a copy of `reviews` with these columns renamed to `region` and `locale`, respectively.

In [ ]:
# Your code here
renamed = reviews.rename(columns ={'region_1':'region','region_2':'locale'})

# Check your answer
q1.check()

In [ ]:
#q1.hint()
#q1.solution()

## 2.
Set the index name in the dataset to `wines`.

In [ ]:
reindexed = reviews.rename_axis("wines", axis='columns')

# Check your answer
q2.check()

In [ ]:
#q2.hint()
#q2.solution()

## 3.
The [Things on Reddit](https://www.kaggle.com/residentmario/things-on-reddit/data) dataset includes product links from a selection of top-ranked forums ("subreddits") on reddit.com. Run the cell below to load a dataframe of products mentioned on the */r/gaming* subreddit and another dataframe for products mentioned on the *r//movies* subreddit.

In [ ]:
gaming_products = pd.read_csv("../input/things-on-reddit/top-things/top-things/reddits/g/gaming.csv")
gaming_products['subreddit'] = "r/gaming"
movie_products = pd.read_csv("../input/things-on-reddit/top-things/top-things/reddits/m/movies.csv")
movie_products['subreddit'] = "r/movies"

gaming_products.head()


In [ ]:
movie_products.head()

Create a `DataFrame` of products mentioned on *either* subreddit.

In [ ]:
combined_products = pd.concat([gaming_products, movie_products])
print(combined_products)
# Check your answer
q3.check()

In [ ]:
#q3.hint()
#q3.solution()

## 4.
The [Powerlifting Database](https://www.kaggle.com/open-powerlifting/powerlifting-database) dataset on Kaggle includes one CSV table for powerlifting meets and a separate one for powerlifting competitors. Run the cell below to load these datasets into dataframes:

In [ ]:
powerlifting_meets = pd.read_csv("../input/powerlifting-database/meets.csv")
powerlifting_competitors = pd.read_csv("../input/powerlifting-database/openpowerlifting.csv")

Both tables include references to a `MeetID`, a unique key for each meet (competition) included in the database. Using this, generate a dataset combining the two tables into one.

In [ ]:
powerlifting_meets  = powerlifting_meets.set_index('MeetID')
powerlifting_competitors = powerlifting_competitors.set_index('MeetID')
powerlifting_combined = powerlifting_competitors.join(powerlifting_meets)

# Check your answer
q4.check()

In [ ]:
# q4.hint()
# q4.solution()

# Congratulations!

You've finished the Pandas micro-course.  Many data scientists feel efficiency with Pandas is the most useful and practical skill they have, because it allows you to progress quickly in any project you have.

If you'd like to apply your new skills to examining geospatial data, you're encouraged to check out our **[Geospatial Analysis](https://www.kaggle.com/learn/geospatial-analysis)** micro-course.

You can also take advantage of your Pandas skills by entering a **[Kaggle Competition](https://www.kaggle.com/competitions)** or by answering a question you find interesting using **[Kaggle Datasets](https://www.kaggle.com/datasets)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/pandas/discussion) to chat with other learners.*